## 3_2_XGBoost 应用

## 3_2_1_XGBoost参数
1. 通用参数：控制整体功能；
2. 提升器参数：在每一步控制单个提升器（tree、regression）；
3. 学习任务参数：控制最优化执行。

### 1.通用参数
booster [default=gbtree]  
选择每次迭代的模型，有两个选择：
+ gbtree：基于树的模型；
+ gbliner：线性模型。  

silent [default=0]
+ 设置为1，静默模式被开启，不会显示运行信息；
+ 通常设置为0，运行信息会更好的帮助理解模型。

nthread [default=最大可能的线程数]
+ 该参数用以并行处理，应设置为系统内核数；
+ 如果你希望使用所有内核，则不应设置该参数，算法会自动检测。

### 2.提升器参数
eta [default=0.3]
+ 学习率；
+ 典型值：0.01-0.2。  

min_child_weight [default=1]
+ 定义最小叶子节点样本权重和；
+ 用于控制过拟合。较大的值可以避免模型学习到局部的特殊样本；
+ 太大的值会导致欠拟合。

max_depth [default=6]
+ 树的最大深度；
+ 用于控制过拟合。较大的值模型会学到更具体更局部的样本；
+ 典型值为3-10。

max_leaf_nodes
+ 树中终端节点或叶子的最大数目；
+ 可以代替max_depth参数。由于创建的是二叉树，一个深度为$n$的树最多生成$2^n$个叶子；
+ 如果该参数被定义，max_depth参数将被忽略。

gamma [default=0]
+ 只有在节点分裂后损失函数值下降，才会分裂该节点。gamma参数指定了节点分裂所需的最小损失函数下降值；
+ 该参数的值越大，算法越保守。该参数的值和损失函数相关，所以是需要调整的。

max_delta_step [default=0]
+ 该参数限制每棵树权重改变的最大步长。如果该参数为0，则表示没有约束。如果将其设置为正值，则使更新步骤更加保守；
+ 通常该参数不需要设置。但是当各类别的样本十分不平衡时，它对逻辑回归是很有帮助的。

subsample [default=1]
+ 该参数控制对于每棵树随机采样的比例；
+ 减小该参数的值，算法会更加保守，避免过拟合。但是，如果该设置得过小，它可能会导致欠拟合；
+ 典型值：0.5-1。

colsample_bytree [default=1]
+ 该参数用来控制每棵随机采样的列数的占比(每一列是一个特征)；
+ 典型值：0.5-1。

colsample_bylevel [default=1]
+ 该参数用来控制树的每一级的每一次分裂，对列数的采样的占比；
+ 该参数和subsample参数可以起到相同的作用。

lambda [default=1]
+ 权重的L2正则化项。(类似于岭回归)。

alpha [default=0]
+ 权重的L1正则化项。(类似于套索回归)；
+ 可以应用在很高维度的情况下，使得算法的速度更快。

scale_pos_weight [default=1]
+ 在各类别样本十分不平衡时，把这个参数设定为一个正值，可以使算法更快收敛。

### 3.学习任务参数
objective [default=reg:linear]  
该参数定义需要被最小化的损失函数。常用值有：
+ binary:logistic 二分类的逻辑回归，返回预测的概率(不是类别)；
+ multi:softmax 使用softmax的多分类器，返回预测的类别(不是概率)。在这种情况下，你还需要多设一个参数：num_class(类别数目)；
+ multi:softprob 和multi:softmax参数一样，但是返回的是每个数据属于各个类别的概率。

eval_metric [ default according to objective ]
+ 对于有效数据的度量方法；
+ 对于回归问题，默认值是rmse，对于分类问题，默认值是error；
+ 典型值：
    + rmse 均方根误差
    + mae 平均绝对误差
    + logloss 负对数似然函数值
    + error 二分类错误率(阈值为0.5)
    + merror 多分类错误率
    + mlogloss 多分类logloss损失函数
    + auc 曲线下面积
    
seed [default=0]
+ 随机数的种子
+ 设置它可以复现随机数据的结果，也可以用于调整参数

## 3_2_2_XGBoost应用

导入XGBoost等相关包：

In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

加载数据，提取特征集和标签：

In [2]:
dataset = loadtxt('./data/pima-indians-diabetes.csv', delimiter=',')

X = dataset[:, 0:8]
y = dataset[:, 8]

将数据划分为训练集和测试集：

In [3]:
seed = 7 
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((514, 8), (254, 8), (514,), (254,))

创建及训练模型：

In [5]:
model = XGBClassifier(n_jobs=-1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

使用训练后的模型对测试集进行预测，并计算预测值与实际之间的acc值：

In [6]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.95%


使用训练后的模型对测试集进行预测，得到每个类别的预测概率：

In [7]:
y_pred_proba = model.predict_proba(X_test)
y_pred_proba

array([[0.9545844 , 0.04541559],
       [0.05245447, 0.9475455 ],
       [0.41897488, 0.5810251 ],
       [0.9831998 , 0.0168002 ],
       [0.4119159 , 0.5880841 ],
       [0.31113452, 0.6888655 ],
       [0.9705527 , 0.02944732],
       [0.93274003, 0.06725994],
       [0.11494881, 0.8850512 ],
       [0.6501156 , 0.34988442],
       [0.03848034, 0.96151966],
       [0.99019825, 0.00980172],
       [0.07478714, 0.92521286],
       [0.0899508 , 0.9100492 ],
       [0.8759558 , 0.12404419],
       [0.8833156 , 0.11668438],
       [0.7805242 , 0.21947582],
       [0.35131902, 0.648681  ],
       [0.98205894, 0.01794105],
       [0.7698676 , 0.23013239],
       [0.6992918 , 0.30070814],
       [0.79318744, 0.20681255],
       [0.3731498 , 0.6268502 ],
       [0.23303777, 0.76696223],
       [0.9244116 , 0.07558843],
       [0.93374586, 0.06625411],
       [0.9396339 , 0.06036608],
       [0.5944243 , 0.4055757 ],
       [0.5389885 , 0.46101153],
       [0.4979669 , 0.5020331 ],
       [0.

In [8]:
y_pred = model.predict(X_test)
y_pred

array([0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 1.

输出各特征重要程度：

In [10]:
from xgboost import plot_importance
from matplotlib import pyplot
%matplotlib inline

plot_importance(model)
pyplot.show()

导入调参相关包：

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

创建模型及参数搜索空间：

In [12]:
model_GS = XGBClassifier()
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
max_depth = [1, 2, 3, 4, 5]
param_grid = dict(learning_rate=learning_rate, max_depth=max_depth)

设置分层抽样验证及创建搜索对象：

In [13]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
grid_search = GridSearchCV(model_GS, param_grid=param_grid, scoring='neg_log_loss', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y)

In [14]:
y_pred = grid_result.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.10%


In [15]:
grid_result.best_score_, grid_result.best_params_

(-0.47171179660714796, {'learning_rate': 0.2, 'max_depth': 1})

## 3_2_3_LightGBM的改进

### 1.直方图算法

<img src="./histogram.jpg" width = "500" height = "300" alt="histogram" />

### 2.树的生成方式  Level-wise VS. Leaf-wise

<img src="./tree.jpg" width = "500" height = "300" alt="tree" />

### 3.并行优化

特征并行：每个worker保存所有数据，在其特征子集上寻找最优切分点，worker之间相互通信，找到全局最优切分点，进行结点分裂；  
数据并行：不同worker合并特征子集的局部直方图，采用直方图做差运算，只需要一个结点直方图数据通信。

### 4.基于梯度的单边采样（Gradient-based One Side Sampling，GOSS）

在每一次迭代前，利用了GBDT中的样本梯度和误差的关系，对训练样本进行采样:   
对误差大（梯度绝对值大）的数据保留；对误差小的数据采样一个子集，但给这个子集的数据一个权重，让这个子集可以近似到误差小的数据的全集。这么采样出来的数据，既不损失误差大的样本，又在减少训练数据的同时不改变数据的分布，从而实现了在几乎不影响精度的情况下加速训练。